<a href="https://colab.research.google.com/github/Sakuni-Weerasinghe/Automatic-Question-and-Answer-Generation-based-on-Large-Language-Models/blob/master/Fluency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install pytorch-lightning

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import random

# Set the seed for reproducibility
random.seed(42)

# Read the dataset from the .txt file
with open('data.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Shuffle the lines randomly
random.shuffle(lines)

# Split the dataset into training and dev sets (e.g., 80% training, 20% dev)
split_ratio = 0.8
split_index = int(len(lines) * split_ratio)

train_set = lines[:split_index]
dev_set = lines[split_index:]

# Write the training set to a new file
with open('train.txt', 'w') as train_file:
    train_file.writelines(train_set)

# Write the dev set to a new file
with open('dev.txt', 'w') as dev_file:
    dev_file.writelines(dev_set)

In [ ]:
!mv train.txt dev.txt drive/MyDrive/Research/QG_Reward/Discriminators/Fluency/Data

In [ ]:
# https://drive.google.com/file/d/1-6Fi6cTheVCfzrR_RTtVblYcAK_gA42R/view?usp=sharing  fluency/dev
# https://drive.google.com/file/d/1-2U39zxPc03k4BlKw9Wu8hXuKPimGvYZ/view?usp=sharing   fluency/train
# https://drive.google.com/file/d/1D74YAZeqCfnhHnBgX_YO2FbR81VbYRmv/view?usp=sharing  fluency/run_model.py

!gdown 'https://drive.google.com/file/d/1-6Fi6cTheVCfzrR_RTtVblYcAK_gA42R' -O dev.txt
!gdown 'https://drive.google.com/file/d/1-2U39zxPc03k4BlKw9Wu8hXuKPimGvYZ' -O train.txt
!gdown 'https://drive.google.com/file/d/1D74YAZeqCfnhHnBgX_YO2FbR81VbYRmv' -O run_model.py

In [ ]:
import os

# Update paths as needed
DATAHOME = "/content/datasets"
EXEHOME = "/content/src"
MODELHOME = "/content/models"

# Make directories
os.makedirs(MODELHOME, exist_ok=True)

# Change directory
os.chdir(EXEHOME)

# Set CUDA_VISIBLE_DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

!python run_model.py \
       --config_name bert-base-uncased \
       --model_type bert \
       --output_dir {MODELHOME} \
       --overwrite_output_dir \
       --tokenizer_name bert-base-uncased \
       --train_data_file {DATAHOME}/train.txt --eval_data_file {DATAHOME}/dev.txt \
       --line_by_line \
       --learning_rate 2e-5 \
       --block_size 128 \
       --per_gpu_train_batch_size 16 \
       --per_gpu_eval_batch_size 8 \
       --do_train \
       --evaluate_during_training \
       --num_train_epochs 10